In [16]:
import os                               
import numpy as np                    
import pandas as pd                     
import matplotlib.pyplot as plt  
import math
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term
from azure.quantum.optimization import ParallelTempering
from azure.quantum.optimization import Term, SlcTerm
import time

Data Preprocessing

In [17]:
stocks = [stock.split('.')[0] for stock in sorted(os.listdir("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)"))]
    
print((stocks))

['ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANIPOWER', 'ADANITRANS', 'ASIANPAINT', 'ATGL', 'AXISBANK', 'BAJAJ-AUTO*', 'BAJAJFINSV', 'BAJFINANCE', 'BHARTIARTL', 'BRITANNIA', 'COALINDIA', 'DABUR', 'DIVISLAB', 'DMART', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HINDUNILVR', 'HINDZINC', 'ICICIBANK', 'INFY', 'IOC', 'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LICI', 'LT', 'M&M', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PIDILITIND', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SBIN', 'SIEMENS', 'SUNPHARMA', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO']


In [18]:
dates = pd.date_range('2022-05-1', '2022-07-01') 
data = pd.DataFrame({'Time': dates})  


In [19]:
for stock in stocks:
    prices = pd.read_csv("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)/"+stock+".csv", usecols=['Date', 'WAP'])
    #print(prices)
    prices['Date'] = pd.to_datetime(prices['Date'], dayfirst = True)  
    #print(prices)
    prices.rename(                                                            
        columns={"Date": "Time", "WAP": stock},
        inplace=True
    )
    data = pd.merge(data,prices)


In [20]:
cp = data .drop(['Time'], axis=1).tail(1).to_numpy()
cp = cp[0]


In [22]:
r = data[(data['Time'] >= '2022-05-01')] \
    .drop(['Time'], axis=1) \
    .pct_change(fill_method='ffill')
r.head()

,ADANIENT,ADANIGREEN,ADANIPORTS,ADANIPOWER,ADANITRANS,ASIANPAINT,ATGL,AXISBANK,BAJAJ-AUTO*,BAJAJFINSV,...,SBIN,SIEMENS,SUNPHARMA,TATAMOTORS,TATASTEEL,TCS,TECHM,TITAN,ULTRACEMCO,WIPRO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.020829,0.047462,0.038108,0.042745,0.039457,0.030649,0.029933,0.019329,0.009675,0.002941,...,0.002107,0.007486,0.018770,0.009478,0.025317,0.018242,0.004394,0.017646,0.026465,0.023720
2,-0.032060,-0.030200,-0.028710,0.014500,-0.043512,-0.016760,-0.053440,-0.017954,-0.015729,-0.038961,...,-0.034241,-0.015113,-0.015054,-0.044467,-0.047224,-0.051670,-0.067516,-0.023153,-0.013995,-0.063767
3,0.033597,0.000165,0.033455,0.045384,0.021460,0.020758,0.009960,0.027695,0.019550,0.016464,...,0.029209,0.025149,0.024132,0.038968,0.024604,0.000889,0.009056,0.004360,0.001613,0.000515
4,0.001730,-0.017945,0.003771,0.048774,-0.036557,0.020568,-0.014307,0.012644,0.004344,0.009640,...,0.011768,0.014560,0.018445,0.017169,-0.112193,0.005359,0.007551,0.017485,-0.006745,0.012204


In [23]:
mu = r.mean().to_numpy()
sigma = r.cov().to_numpy()
n=len(stocks)


In [24]:
'''from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "e87e6b08-4913-4522-9206-ba18a56ee0fa",
    resource_group = "AzureQuantum",
    name = "MTCProjectQuantumSharique",
    location = "East US"
)
'''

from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "6e02aef9-2670-4418-92b8-7659cf5605d2",
    resource_group = "azurequantum",
    name = "workspace1",
    location = "japaneast"
)

Objective Function

In [25]:
 
def Return(mu,n):
    """Calculates the Terms for the return component of the cost function

    Parameters
    ----------
    mu : np.array
        Expected return of each stock 
    n : integer
        Number of stocks
        
    Returns
    -------
    terms: list
        List of Terms associated with the return component of the cost function
    """
    
    terms=[]
    
    for i in range(0,n):
        
        terms.append(Term(c=-1*mu[i] , indices=[i]))
    print("Number of terms in return: ", len(terms))
    return terms

In [26]:
def risk(sigma,n,risk_aversion):

    """Calculates the Terms for the risk component of the cost function

    Parameters
    ----------
    sigma : np.array
        Covariance matrix of the dataset 
    n : integer
        Number of stocks
    risk_aversion: integer
        Risk aversion parameter
        
    Returns
    -------
    terms: list
        List of Terms associated with the risk component of the cost function
    """
    
    terms=[]
    
    for i in range(0,n):
        for j in range(0,n):
            terms.append(Term(c= sigma[i][j] * (risk_aversion/2) , indices=[i,j]))
    print("Number of terms in risk: ", len(terms))
    return terms

Constraints

In [27]:
def budget_constraint(penalty_weight,n,b):

    '''
      Calculates the Terms for the normalized budget constraint

        Parameters
        ----------
        penalty_weight : integer
            Lagrange multiplier to penalise the stocks that don't obey the constraint 
        n : integer
            Number of stocks
        b : integer
            Normalized budget / stock appretite must be less than n

        Returns
        -------
        slc: list
            List of SlcTerms associated with the penalty constraint
        
    '''

    terms=[]
    for i in range(0,n):
        terms.append(Term(c= 1 , indices=[i]))
    terms.append(Term(c=-b, indices=[]))
    slc = []
    slc.append(SlcTerm(
            terms,
            c=1
        ))
    return slc

Results

In [28]:
terms=[]
t1 = Return(mu,n)
t2 = risk(sigma,n,1)
terms= t1+ t2 
problem = Problem(name="Portfolio optimization", problem_type=ProblemType.pubo, terms=terms)

Number of terms in return:  50
Number of terms in risk:  2500


In [29]:
solver = ParallelTempering(workspace)
solution = solver.optimize(problem)

....

In [30]:
print(solution['configuration'])
result = solution['configuration']
#print(type(result))

selected_stocks=[]
prices_selected=[]
index_prices_selected= []

daily_prices = data.drop(['Time'], axis=1).to_numpy()
#print(daily_prices)

pmu = []
for i in range(0,34):
    for j in range(0,n):
        pmu.append(daily_prices[i][j].mean()) 
#print(pmu)

for i in result.keys():
    if result[i]:
        selected_stocks.append(stocks[int(i)])
        prices_selected.append(pmu[int(i)])
        index_prices_selected.append(int(i))
        
print("\nSelected stocks are -\n", selected_stocks)

print("\nPrice of selected stocks are -\n", prices_selected)

{'0': 0, '1': 0, '2': 0, '3': 0, '4': 1, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 1, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 1, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 1, '27': 0, '28': 0, '29': 0, '30': 0, '31': 1, '32': 1, '33': 1, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 1, '40': 0, '41': 1, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47': 0, '48': 0, '49': 0}

Selected stocks are -
 ['ADANITRANS', 'BRITANNIA', 'HDFCBANK', 'ITC', 'M&M', 'MARUTI', 'NESTLEIND', 'SBILIFE', 'SIEMENS']

Price of selected stocks are -
 [2288.050953422945, 3353.182279766438, 1307.06281447588, 262.41769359302384, 903.5962191292541, 7478.342472415576, 16291.96707229778, 1058.2533124128313, 2316.5616353863775]


In [31]:
def find_risk():

    """Calculates the risk for the optimised portfolio
    
    Parameters
    ----------
    None

    Returns
    -------
    risk: float
        risk value of the optimised portfolio
    """
    
    risk=0
    for i in range(0,n):
        for j in range(0,n):
            t = 0.5 * result[str(i)]* result[str(j)]*sigma[i][j]
            risk+=t
    return risk

In [32]:

def distributed_budget(B, arr, cp):

    '''
    Distributes the budget equally among all stocks to calculate the number of shares of each stock

    Parameters-
        B - Budget to be invested
        arr- List of strings representing selected stocks 
        cp - A numpy matrix representing the current prices

    Returns-
        shares - list containing number of shares of each stock to select
        
    '''

    shares=[]
    distribution = B/len(arr)  

    for i in range(len(arr)):
        numberOfShares = math.floor(distribution/cp[index_prices_selected[i]])   
        shares.append(numberOfShares)
        print(arr[i], " : " ,shares[i]," shares")
    return shares

In [34]:
no_of_shares = distributed_budget(200000, selected_stocks, cp)
#type(no_of_shares[0])

ADANITRANS  :  8  shares
BRITANNIA  :  6  shares
HDFCBANK  :  16  shares
ITC  :  79  shares
M&M  :  20  shares
MARUTI  :  2  shares
NESTLEIND  :  1  shares
SBILIFE  :  20  shares
SIEMENS  :  9  shares


In [35]:

def find_return():
    '''
    Calculates the expected return for the optimised portfolio
    
    Parameters
    ----------
    None

    Returns
    -------
    Return: float
        Expected return value of the optimised portfolio
    
    '''
    Return=0
    number_stocks = len(selected_stocks)
    for i in range(0,number_stocks):
        t = prices_selected[i]*number_stocks
        Return+=t
    return Return

In [37]:
# Printing results

print("OPTIMIZSED PORTFOLIO : \n")
portfolio= distributed_budget(200000, selected_stocks, cp)
print("\nCONSTRAINTS:")
print("\nBudget : ",  u"\u20B9", 200000,"\n")
print("Risk Aversion index of :" , 1)
print("\nReturns : ",  u"\u20B9", round(find_return(),3))
print("\nRisk : ", find_risk()*100,"%")

OPTIMIZSED PORTFOLIO : 

ADANITRANS  :  8  shares
BRITANNIA  :  6  shares
HDFCBANK  :  16  shares
ITC  :  79  shares
M&M  :  20  shares
MARUTI  :  2  shares
NESTLEIND  :  1  shares
SBILIFE  :  20  shares
SIEMENS  :  9  shares

CONSTRAINTS:

Budget :  ₹ 200000 

Risk Aversion index of : 1

Returns :  ₹ 317334.91

Risk :  0.4000232128394136 %


In [38]:
terms=[]
t1 = Return(mu,n)
t2 = risk(sigma,n,1)
t3 = budget_constraint(2*n,n,8)
terms= t1+ t2 + t3
problem = Problem(name="Portfolio optimization", problem_type=ProblemType.pubo, terms=terms)

Number of terms in return:  50
Number of terms in risk:  2500


In [39]:
# Instantiate a solver to solve the problem. 
solver = ParallelTempering(workspace)

# Optimize the problem
solution = solver.optimize(problem)

......

In [40]:
print(solution['configuration'])
result = solution['configuration']
#print(type(result))

selected_stocks=[]
prices_selected=[]
index_prices_selected= []

daily_prices = data.drop(['Time'], axis=1).to_numpy()
#print(daily_prices)

pmu = []
for i in range(0,34):
    for j in range(0,n):
        pmu.append(daily_prices[i][j].mean()) 
#print(pmu)

for i in result.keys():
    if result[i]:
        selected_stocks.append(stocks[int(i)])
        prices_selected.append(pmu[int(i)])
        index_prices_selected.append(int(i))
        
print("\nSelected stocks are -\n", selected_stocks)

print("\nPrice of selected stocks are -\n", prices_selected)

{'0': 1, '1': 0, '2': 0, '3': 0, '4': 1, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 1, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 1, '27': 0, '28': 0, '29': 0, '30': 0, '31': 1, '32': 1, '33': 1, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 1, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47': 0, '48': 0, '49': 0}

Selected stocks are -
 ['ADANIENT', 'ADANITRANS', 'HDFCLIFE', 'ITC', 'M&M', 'MARUTI', 'NESTLEIND', 'SBILIFE']

Price of selected stocks are -
 [2148.289147665696, 2288.050953422945, 549.7553284184845, 262.41769359302384, 903.5962191292541, 7478.342472415576, 16291.96707229778, 1058.2533124128313]


In [44]:
# Printing results

print("\n DIVERSIFIED PORTFOLIO: \n")
ab= distributed_budget(200000, selected_stocks, cp)
print("\nConstraints:")
print("\nBudget : ",  u"\u20B9", 200000,"\n")
print("Stock Appetite: ", 13, " stocks\n")
print("Risk Aversion index of :" , 1,"\n")
print("Returns : ",  u"\u20B9", round(find_return(),3))
print("\nRisk : ", round(find_risk()*100,4) ,"%")


 DIVERSIFIED PORTFOLIO: 

ADANIENT  :  11  shares
ADANITRANS  :  10  shares
HDFCLIFE  :  44  shares
ITC  :  89  shares
M&M  :  22  shares
MARUTI  :  2  shares
NESTLEIND  :  1  shares
SBILIFE  :  22  shares

Constraints:

Budget :  ₹ 200000 

Stock Appetite:  13  stocks

Risk Aversion index of : 1 

Returns :  ₹ 247845.378

Risk :  0.4213 %
